In [1]:
import gzip
import os
import tempfile
import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [2]:
def get_files(filename):
    class_train = []
    label_train = []
    for train_class in os.listdir(filename):
        for pic in os.listdir(filename+train_class):
            class_train.append(filename+train_class+'\\'+pic)
            label_train.append(train_class)
    temp = np.array([class_train,label_train])
    temp = temp.transpose()
    #shuffle the samples
    np.random.shuffle(temp)
    #after transpose, images is in dimension 0 and label in dimension 1
    image_list = list(temp[:,0])
    label_list = list(temp[:,1])
    label_list = [int(i) for i in label_list]
    #print(image_list)
    #print(label_list)
    return image_list,label_list

In [3]:
def get_batches(image,label,resize_w,resize_h,batch_size,capacity):
    #convert the list of images and labels to tensor
    image = tf.cast(image,tf.string)
    label = tf.cast(label,tf.int64)
    queue = tf.train.slice_input_producer([image,label])
    label = queue[1]
    image_c = tf.read_file(queue[0])
    image = tf.image.decode_jpeg(image_c,channels = 3)
    #resize
    image = tf.image.resize_image_with_crop_or_pad(image,resize_w,resize_h)
    #(x - mean) / adjusted_stddev
    image = tf.image.per_image_standardization(image)
    
    image_batch,label_batch = tf.train.batch([image,label],
                                             batch_size = batch_size,
                                             num_threads = 64,
                                             capacity = capacity)
    images_batch = tf.cast(image_batch,tf.float32)
    labels_batch = tf.reshape(label_batch,[batch_size])
    return images_batch,labels_batch

In [4]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev = 0.01))
#init weights
weights = {
    "w1":init_weights([5,5,3,16]),
    "w2":init_weights([5,5,16,32]),
    "w3":init_weights([5,5,32,64]),
    "w4":init_weights([5,5,64,128]),
    #"w3":init_weights([3,3,32,128]),
    "w5":init_weights([32768,1024]),
    "wo":init_weights([1024,6])
    }

#init biases
biases = {
    "b1":init_weights([16]),
    "b2":init_weights([32]),
    "b3":init_weights([64]),
    "b4":init_weights([128]),
    #"b3":init_weights([256]),
    "b5":init_weights([1024]),
    "bo":init_weights([6])
    }

# defining conv and pooling
def conv2d(x,w,b):
    x = tf.nn.conv2d(x,w,strides = [1,1,1,1],padding = "SAME")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def pooling(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],padding = "SAME")

def norm(x,lsize = 4):
    return tf.nn.lrn(x,depth_radius = lsize,bias = 1,alpha = 0.001/9.0,beta = 0.75)


In [5]:
def mmodel(images):
    l1 = conv2d(images,weights["w1"],biases["b1"])
    l2 = pooling(l1)
    l2 = norm(l2)
    l3 = conv2d(l2,weights["w2"],biases["b2"])
    l4 = pooling(l3)
    l4 = norm(l4)
    l5 = conv2d(l4,weights["w3"],biases["b3"])
    l6 = pooling(l5)
    l6 = norm(l6)
    l7 = conv2d(l6,weights["w4"],biases["b4"])
    l8 = pooling(l7)
    l8 = norm(l8)
    #l4 = norm(l4)
    #l5 = conv2d(l4,weights["w3"],biases["b3"])
    #same as the batch size
    #l6 = pooling(l5)
    l9 = tf.reshape(l8,[-1,weights["w5"].get_shape().as_list()[0]])
    l10 = tf.nn.relu(tf.matmul(l9,weights["w5"])+biases["b5"])
    l10 = tf.nn.dropout(l10,0.8)
    soft_max = tf.add(tf.matmul(l10,weights["wo"]),biases["bo"])
    return soft_max

In [6]:
def loss(logits,label_batches):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=label_batches)
    cost = tf.reduce_mean(cross_entropy)
    return cost

In [7]:
def training(loss,lr):
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)
    return train_op

In [8]:
def get_accuracy(logits,labels):
    acc = tf.nn.in_top_k(logits,labels,1)
    acc = tf.cast(acc,tf.float32)
    acc = tf.reduce_mean(acc)
    return acc

In [9]:
#def run_training():
data_dir = 'C:\\Users\\Leonard\\Desktop\\data\\'
image,label = get_files(data_dir)
#print(image)
print(label)
image_batches,label_batches = get_batches(image,label,256,256,20,100)
p = mmodel(image_batches)
print(p)
cost = loss(p,label_batches)
train_op = training(cost,0.0001)
acc = get_accuracy(p,label_batches)
 
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess,coord = coord)
accs = 0

try:
    for step in np.arange(1500):
        print(step)
        if coord.should_stop():
            break
        _,train_acc,train_loss = sess.run([train_op,acc,cost])
        print("loss:{} accuracy:{}".format(train_loss,train_acc))
        accs = accs + train_acc
        if step%50 == 0:
            acca = accs / 50
            print("average acc:{}".format(acca))
            accs = 0
            acca = 0
            
        
except tf.errors.OutOfRangeError:
    print("Done!!!")
finally:
    coord.request_stop()
    coord.join(threads)
    sess.close() 

[0, 0, 0, 0, 2, 2, 3, 5, 1, 1, 0, 2, 5, 5, 0, 4, 2, 2, 1, 4, 0, 1, 5, 3, 2, 3, 4, 0, 0, 2, 1, 4, 2, 4, 5, 0, 2, 2, 0, 4, 4, 0, 2, 2, 1, 3, 2, 0, 3, 2, 5, 3, 2, 3, 2, 3, 2, 2, 5, 2, 3, 1, 3, 2, 5, 5, 4, 5, 0, 0, 1, 5, 3, 4, 2, 5, 0, 1, 2, 2, 5, 1, 1, 5, 5, 2, 1, 4, 5, 4, 1, 2, 4, 2, 3, 0, 2, 0, 3, 0, 1, 4, 2, 5, 1, 0, 0, 4, 0, 4, 5, 0, 4, 0, 1, 0, 3, 5, 3, 2, 0, 4, 5, 2, 2, 0, 1, 5, 3, 3, 2, 3, 1, 4, 2, 3, 4, 0, 5, 4, 4, 3, 5, 3, 1, 4, 2, 0, 2, 3, 3, 0, 4, 2, 4, 3, 1, 5, 2, 3, 0, 0, 0, 5, 1, 5, 4, 4, 2, 1, 3, 5, 0, 4, 3, 1, 2, 2, 3, 0, 4, 3, 2, 2, 3, 2, 0, 4, 4, 5, 5, 2, 2, 0, 5, 2, 0, 1, 5, 5, 3, 2, 5, 1, 1, 2, 4, 3, 2, 4, 4, 0, 2, 5, 2, 4, 4, 0, 4, 5, 3, 4, 2, 5, 4, 4, 5, 5, 1, 5, 3, 1, 1, 4, 5, 1, 4, 1, 3, 2, 1, 4, 4, 0, 2, 4, 4, 3, 2, 5, 3, 1, 0, 1, 3, 3, 1, 4, 1, 4, 5, 4, 1, 5, 1, 5, 0, 4, 5, 3, 4, 5, 1, 3, 5, 1, 5, 0, 4, 3, 0, 3, 2, 1, 3, 4, 0, 1, 4, 2, 0, 1, 0, 5, 4, 5, 5, 5, 5, 1, 4, 4, 1, 1, 5, 4, 5, 5, 2, 3, 0, 0, 0, 3, 2, 0, 0, 4, 1, 2, 3, 5, 3, 0, 5, 0, 3, 2, 3, 2, 1, 2, 3, 

loss:1.099115252494812 accuracy:0.550000011920929
87
loss:1.3998229503631592 accuracy:0.45000001788139343
88
loss:1.205629587173462 accuracy:0.5
89
loss:1.2983362674713135 accuracy:0.44999998807907104
90
loss:1.3149209022521973 accuracy:0.5
91
loss:1.2222387790679932 accuracy:0.6000000238418579
92
loss:1.1429660320281982 accuracy:0.550000011920929
93
loss:1.104560375213623 accuracy:0.6000000238418579
94
loss:1.4820120334625244 accuracy:0.25
95
loss:1.4519544839859009 accuracy:0.3500000238418579
96
loss:1.1555455923080444 accuracy:0.4000000059604645
97
loss:1.3513420820236206 accuracy:0.4000000059604645
98
loss:1.4261788129806519 accuracy:0.44999998807907104
99
loss:1.1275888681411743 accuracy:0.550000011920929
100
loss:1.340735912322998 accuracy:0.550000011920929
average acc:0.428000006377697
101
loss:1.1553847789764404 accuracy:0.5
102
loss:1.2523870468139648 accuracy:0.550000011920929
103
loss:1.2817955017089844 accuracy:0.5
104
loss:1.6268556118011475 accuracy:0.20000000298023224
10

loss:1.1134262084960938 accuracy:0.550000011920929
241
loss:1.1285812854766846 accuracy:0.550000011920929
242
loss:1.0728518962860107 accuracy:0.6500000357627869
243
loss:1.1179028749465942 accuracy:0.6000000238418579
244
loss:0.9007685780525208 accuracy:0.7000000476837158
245
loss:1.0734989643096924 accuracy:0.550000011920929
246
loss:1.4416053295135498 accuracy:0.3499999940395355
247
loss:1.0157294273376465 accuracy:0.550000011920929
248
loss:1.0653564929962158 accuracy:0.6499999761581421
249
loss:1.2164056301116943 accuracy:0.44999998807907104
250
loss:1.1163326501846313 accuracy:0.6000000238418579
average acc:0.5230000108480454
251
loss:1.0856642723083496 accuracy:0.44999998807907104
252
loss:1.1867170333862305 accuracy:0.5
253
loss:1.1417133808135986 accuracy:0.6000000238418579
254
loss:1.1617263555526733 accuracy:0.5
255
loss:0.8345938324928284 accuracy:0.6499999761581421
256
loss:1.1690638065338135 accuracy:0.550000011920929
257
loss:1.4065511226654053 accuracy:0.449999988079071

loss:1.1051743030548096 accuracy:0.44999998807907104
396
loss:1.1680206060409546 accuracy:0.550000011920929
397
loss:1.1938467025756836 accuracy:0.4000000059604645
398
loss:1.0407344102859497 accuracy:0.550000011920929
399
loss:0.8978524208068848 accuracy:0.7000000476837158
400
loss:1.030538558959961 accuracy:0.6500000357627869
average acc:0.6080000096559525
401
loss:0.9993765354156494 accuracy:0.550000011920929
402
loss:1.030316948890686 accuracy:0.6000000238418579
403
loss:0.9992746114730835 accuracy:0.6499999761581421
404
loss:1.0958311557769775 accuracy:0.5
405
loss:1.1680164337158203 accuracy:0.6500000357627869
406
loss:0.8353396654129028 accuracy:0.7000000476837158
407
loss:0.9405144453048706 accuracy:0.6499999761581421
408
loss:1.1537132263183594 accuracy:0.550000011920929
409
loss:1.111894965171814 accuracy:0.550000011920929
410
loss:1.1778925657272339 accuracy:0.5
411
loss:1.0895366668701172 accuracy:0.5
412
loss:1.03232741355896 accuracy:0.6000000238418579
413
loss:1.35665607

loss:0.4324140250682831 accuracy:0.8500000238418579
550
loss:0.46231287717819214 accuracy:0.8500000238418579
average acc:0.7150000095367431
551
loss:0.5143526792526245 accuracy:0.75
552
loss:0.8497779369354248 accuracy:0.7000000476837158
553
loss:0.7520272731781006 accuracy:0.6500000357627869
554
loss:0.7217926979064941 accuracy:0.6499999761581421
555
loss:0.6058657765388489 accuracy:0.75
556
loss:1.0526299476623535 accuracy:0.5
557
loss:0.6692901849746704 accuracy:0.800000011920929
558
loss:1.0678620338439941 accuracy:0.6499999761581421
559
loss:0.8158944249153137 accuracy:0.75
560
loss:0.6536545157432556 accuracy:0.75
561
loss:0.8078881502151489 accuracy:0.75
562
loss:0.9839249849319458 accuracy:0.550000011920929
563
loss:0.6210548877716064 accuracy:0.800000011920929
564
loss:0.378550261259079 accuracy:0.8999999761581421
565
loss:0.6099609136581421 accuracy:0.7000000476837158
566
loss:0.7407997250556946 accuracy:0.7000000476837158
567
loss:1.2032153606414795 accuracy:0.35000002384185

...

loss:0.2477266788482666 accuracy:0.8500000238418579
858
loss:0.1832241415977478 accuracy:1.0
859
loss:0.318725049495697 accuracy:0.8500000238418579
860
loss:0.38183021545410156 accuracy:0.75
861
loss:0.17925351858139038 accuracy:0.8999999761581421
862
loss:0.3018365502357483 accuracy:0.8500000238418579
863
loss:0.1852499544620514 accuracy:0.8999999761581421
864
loss:0.35430508852005005 accuracy:0.8999999761581421
865
loss:0.2354229986667633 accuracy:0.8500000238418579
866
loss:0.34198153018951416 accuracy:0.8999999761581421
867
loss:0.26795753836631775 accuracy:0.8500000238418579
868
loss:0.29900142550468445 accuracy:0.8500000238418579
869
loss:0.27119120955467224 accuracy:0.8500000238418579
870
loss:0.21674270927906036 accuracy:0.8999999761581421
871
loss:0.29571154713630676 accuracy:0.8500000238418579
872
loss:0.6825131177902222 accuracy:0.7000000476837158
873
loss:0.559604823589325 accuracy:0.800000011920929
874
loss:0.14653100073337555 accuracy:0.949999988079071
875
loss:0.22574087

loss:0.2533029317855835 accuracy:0.8999999761581421
1006
loss:0.25786522030830383 accuracy:0.8999999761581421
1007
loss:0.42957597970962524 accuracy:0.800000011920929
1008
loss:0.4212571084499359 accuracy:0.8500000238418579
1009
loss:0.32836323976516724 accuracy:0.8500000238418579
1010
loss:0.18098801374435425 accuracy:0.949999988079071
1011
loss:0.2691554129123688 accuracy:0.8500000238418579
1012
loss:0.15425892174243927 accuracy:0.949999988079071
1013
loss:0.22883787751197815 accuracy:0.949999988079071
1014
loss:0.0974883884191513 accuracy:1.0
1015
loss:0.407431423664093 accuracy:0.800000011920929
1016
loss:0.48740750551223755 accuracy:0.800000011920929
1017
loss:0.25494685769081116 accuracy:0.8999999761581421
1018
loss:0.48742222785949707 accuracy:0.800000011920929
1019
loss:0.18893615901470184 accuracy:0.949999988079071
1020
loss:0.11324663460254669 accuracy:0.949999988079071
1021
loss:0.07555045187473297 accuracy:1.0
1022
loss:0.15077471733093262 accuracy:0.8999999761581421
1023
l

loss:0.08250471949577332 accuracy:0.949999988079071
1156
loss:0.17567428946495056 accuracy:0.8500000238418579
1157
loss:0.21408352255821228 accuracy:0.8500000238418579
1158
loss:0.13974621891975403 accuracy:0.949999988079071
1159
loss:0.19050440192222595 accuracy:0.8999999761581421
1160
loss:0.08479263633489609 accuracy:0.949999988079071
1161
loss:0.15506663918495178 accuracy:0.949999988079071
1162
loss:0.32723164558410645 accuracy:0.800000011920929
1163
loss:0.37764132022857666 accuracy:0.800000011920929
1164
loss:0.25379127264022827 accuracy:0.8500000238418579
1165
loss:0.16103145480155945 accuracy:0.8999999761581421
1166
loss:0.29440492391586304 accuracy:0.8500000238418579
1167
loss:0.25620877742767334 accuracy:0.8500000238418579
1168
loss:0.22129443287849426 accuracy:0.949999988079071
1169
loss:0.21725668013095856 accuracy:0.8999999761581421
1170
loss:0.38055676221847534 accuracy:0.75
1171
loss:0.25719523429870605 accuracy:0.8999999761581421
1172
loss:0.2374993860721588 accuracy:0.

loss:0.34472623467445374 accuracy:0.800000011920929
1302
loss:0.23346489667892456 accuracy:0.8500000238418579
1303
loss:0.12035906314849854 accuracy:0.949999988079071
1304
loss:0.21495912969112396 accuracy:0.8999999761581421
1305
loss:0.3001681864261627 accuracy:0.800000011920929
1306
loss:0.16288350522518158 accuracy:0.8999999761581421
1307
loss:0.3077969253063202 accuracy:0.800000011920929
1308
loss:0.16026517748832703 accuracy:0.8999999761581421
1309
loss:0.21912340819835663 accuracy:0.8999999761581421
1310
loss:0.07973597943782806 accuracy:1.0
1311
loss:0.2791353464126587 accuracy:0.8999999761581421
1312
loss:0.102139413356781 accuracy:0.949999988079071
1313
loss:0.2472759187221527 accuracy:0.8500000238418579
1314
loss:0.39569905400276184 accuracy:0.800000011920929
1315
loss:0.050999365746974945 accuracy:1.0
1316
loss:0.2761842608451843 accuracy:0.8999999761581421
1317
loss:0.24830403923988342 accuracy:0.8500000238418579
1318
loss:0.30582019686698914 accuracy:0.800000011920929
1319

loss:0.2074357569217682 accuracy:0.8500000238418579
1450
loss:0.12269812822341919 accuracy:0.8999999761581421
average acc:0.8949999976158142
1451
loss:0.0754764974117279 accuracy:1.0
1452
loss:0.17744305729866028 accuracy:0.949999988079071
1453
loss:0.21768218278884888 accuracy:0.8500000238418579
1454
loss:0.20700328052043915 accuracy:0.8500000238418579
1455
loss:0.13966333866119385 accuracy:0.949999988079071
1456
loss:0.16726979613304138 accuracy:0.8999999761581421
1457
loss:0.2994646430015564 accuracy:0.800000011920929
1458
loss:0.2902774214744568 accuracy:0.8999999761581421
1459
loss:0.18951883912086487 accuracy:0.8999999761581421
1460
loss:0.15855813026428223 accuracy:0.8500000238418579
1461
loss:0.09344515204429626 accuracy:0.949999988079071
1462
loss:0.044224608689546585 accuracy:1.0
1463
loss:0.012731345370411873 accuracy:1.0
1464
loss:0.11984636634588242 accuracy:0.8999999761581421
1465
loss:0.18320894241333008 accuracy:0.949999988079071
1466
loss:0.3079928755760193 accuracy:0.